In [57]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
import matplotlib.pyplot as plt
import PIL
from importlib import import_module
from pathlib import Path
from distutils import util

import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_pil_image
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

In [58]:
data_transform = transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
])

training_data = datasets.CIFAR10(
    root="data", # Download True일 시, 데이터를 다운받을 경로 // Download False일 시, 데이터가 존재하는 경로
    train=True,  # Training data
    download=False,
    transform=data_transform,
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.CIFAR10(
    root="data",
    train=False, # Test data
    download=False,
    transform=data_transform,
)

batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(dataset=training_data,
    batch_size=batch_size,
    shuffle=True)
test_dataloader = DataLoader(dataset=test_data, 
    batch_size=batch_size,
    shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 3, 227, 227])
Shape of y: torch.Size([64]) torch.int64


In [59]:
class AlexNet(nn.Module):
    """
    Neural network model consisting of layers propsed by AlexNet paper.
    """
    def __init__(self, num_classes=10):
        """
        Define and allocate layers for this neural net.
        Args:
            num_classes (int): number of classes to predict with this model
        """
        super().__init__()
        # input size should be : (b x 3 x 227 x 227)
        # The image in the original paper states that width and height are 224 pixels, but
        # the dimensions after first convolution layer do not lead to 55 x 55.
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        self.init_bias()  # initialize bias

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[4].bias, 1)
        nn.init.constant_(self.net[10].bias, 1)
        nn.init.constant_(self.net[12].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 256 * 6 * 6)  # reduce the dimensions for linear layer input
        return self.classifier(x)

In [60]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)

# -- settings

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# -- dataset & augmentation

data_transform = transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.CIFAR10(
    root="data", # Download True일 시, 데이터를 다운받을 경로 // Download False일 시, 데이터가 존재하는 경로
    train=True,  # Training data
    download=False,
    transform=data_transform,
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.

test_data = datasets.CIFAR10(
    root="data",
    train=False, # Test data
    download=False,
    transform=data_transform,
)


# 데이터로더를 생성합니다.

batch_size = 64

train_dataloader = DataLoader(dataset=training_data,
    batch_size=batch_size,
    pin_memory=use_cuda,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=True)
    
test_dataloader = DataLoader(dataset=test_data,
    batch_size=batch_size,
    pin_memory=use_cuda,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



Shape of X [N, C, H, W]: torch.Size([64, 3, 227, 227])
Shape of y: torch.Size([64]) torch.int64


In [61]:
# -- model

model = AlexNet().to(device)
model = torch.nn.DataParallel(model)

print(model)

DataParallel(
  (module): AlexNet(
    (net): Sequential(
      (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
      (1): ReLU()
      (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): ReLU()
      (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU()
      (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU()
      (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): 

In [62]:
optimizer = optim.SGD(
        params=model.parameters(),
        lr=0.01,
        momentum=0.9,
        weight_decay=0.0005)

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [66]:
# Training

torch.cuda.empty_cache()

for epoch in range(100):
    model.train()

    loss_value = 0
    matches = 0

    for idx, train_batch in enumerate(train_dataloader):
        lr_scheduler.step()

        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = F.cross_entropy(outs, labels)

        loss.backward()
        optimizer.step()

        if idx % 100 == 0:
            loss, current = loss.item(), idx * len(X)
            size = len(train_dataloader.dataset)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        size = len(test_dataloader.dataset)
        num_batches = len(test_dataloader)

        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            test_pred = model(X)
            test_loss += F.cross_entropy(test_pred, y).item()
            correct += (torch.argmax(test_pred, dim=-1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


loss: 2.302752  [    0/50000]
loss: 2.300008  [ 6400/50000]
loss: 2.296355  [12800/50000]
loss: 2.294404  [19200/50000]
loss: 2.309405  [25600/50000]
loss: 2.301866  [32000/50000]
loss: 2.311103  [38400/50000]
loss: 2.296681  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302927 

loss: 2.307286  [    0/50000]
loss: 2.301385  [ 1600/50000]
loss: 2.303610  [ 3200/50000]
loss: 2.304586  [ 4800/50000]
loss: 2.308570  [ 6400/50000]
loss: 2.301448  [ 8000/50000]
loss: 2.304954  [ 9600/50000]
loss: 2.309008  [11200/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302927 

loss: 2.307623  [    0/50000]


KeyboardInterrupt: 

In [38]:
(torch.argmax(torch.Tensor([1,2,3,1]), dim=-1)==torch.Tensor([1,2,3,4,5,6,7])).type(torch.float).sum()

tensor(1.)